In [ ]:
import os
import sys
sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use("bmh")
%matplotlib inline
import missingno as msno
import itertools
from multiprocessing import Pool

from speclib.loaders import loadUser, loadUserPhonenumberDict, getUserList, Useralias, loadUserParallel
from speclib.plotting import looseAxesLimits

In [ ]:
userLst = getUserList() 

In [ ]:
useralias = Useralias() 

In [ ]:
userSpec = [(username, useralias[username], ('call', 'sms')) for username in userLst]
for el in userSpec[:10]:
    print(el)

In [ ]:
users = loadUserParallel(userSpec, 15) 

In [ ]:
def dict2DataFrame(dct):
    df = pd.DataFrame(dct) 
    if 'id' in df.columns:
        df = df.drop("id", axis=1)
    if 'type' in df.columns:
        df = df.drop('type', axis=1)
    df.timestamp = df.timestamp.astype('datetime64[s]')  # convert Unix integer timestamps to datetime-objects
    df.user = df.user.apply(lambda x: useralias[x])  # convert hex-usernames to enumerated usernames
    if 'address' in df.columns:
        df["comtype"] = "sms"  # create the column "comtype", set it to "sms"
        df.rename(columns={'address': 'number'}, inplace=True)
    else:
        df["comtype"] = "call"
    df = df.set_index(["user", "comtype"])  # move columns "user" and "comtype" to (multi)index
    return df

In [ ]:
df1c = dict2DataFrame(users["u0001"]['call'])
df1s = dict2DataFrame(users["u0001"]['sms'])
df2c = dict2DataFrame(users["u0002"]['call'])
df2s = dict2DataFrame(users["u0002"]['sms']) 

In [ ]:
from IPython.display import display
display(df1c.columns, df1s.columns)

In [ ]:
def users2DataFrame(dct, processes=15):
    gen = (comDct for user in dct.values()
                      for comDct in user.values() if comDct)
    try:
        pool = Pool(processes=processes)
        call = pool.map_async(dict2DataFrame, gen)
        call.wait()
        toConcatenate = call.get()
    finally:
        pool.terminate()
    return toConcatenate

In [ ]:
tc = users2DataFrame(users) 

In [ ]:
df = pd.concat(tc)

In [ ]:
df

In [ ]:
idx = df.loc["u0645", "sms"].body == "cc0bf55fbc000c9ffa5ca348a1724744ae704ae0"

In [ ]:
df.loc["u0645", "sms"][idx]